In [1]:
import re
from itertools import product
from pprint import pprint
from random import choice

# Supplementary code for the #papername

## Data generators

The first part of this notebook contains code for test set generation.

* **First-last harmony** is a harmony where the first and the last elements of the string are vowels, and they need to agree. For example, `axoxoa` is a good word, while `axaxao` is not.
* **Double harmony** is when a vowel harmony and a consonant harmony co-occur within the same language. For example, `apapp`, `abbap` and `popooo` are good, while `apabo` and `appoppp` are not.
* **One or two locally-dependent simultaneous assimilations** where the trigger for a long-distance assimilation depends on a local context. For example, `e` immediately before `x` prohibits `a` anywhere further after `e` in the string. Then `eaaxaae` and `axaexeeexx` are good, while `exxae` is not.

### First-last harmony
The first and the last elements of the string are vowels, and they need to agree. For example, `axoxoa` is a good word, while `axaxao` is not.

In [2]:
def first_last_generate(n = 10, length = 10, grammatical = True, 
                        vowels = None, transparent = None):
    """ Generates a collection words following the rule of first-last harmony.
    
    * n (int): number of strings that need to be generated;
    * length (int): length of every one of the generated strings;
    * grammatical (bool): if set to True, the correctly harmonizing
                          forms are generated, and if set to False,
                          the disharmonic forms are produced;
    * vowels (list): list of vowels among which the first-last
                     agreement is established;
    * transparent (list): list of irrelevant elements.
    """
    
    # initialization and sanity check for the list of vowels
    if vowels is None:
        vowels = ["a", "o"]
    elif len(vowels) < 2:
        raise IndexError("The vowel system needs to contain at least two distinct vowels.")
    
    # initialization and sanity check for the list of transparent elements
    if transparent is None:
        transparent = ["x"]
    elif [i for i in vowels if i in transparent]:
        raise ValueError("Lists of harmonizing vowels and transparent elements cannot overlap.")
    
    # generate the required number of harmonic strings
    strings = []
    for i in range(n):
        new = choice(vowels)
        new += "".join([choice(vowels + transparent) for j in range(length - 2)])
        if grammatical:
            new += new[0]
        else:
            new += choice([i for i in vowels if i != new[0]])
            
        strings.append(new)

    return strings

### Double harmony

When a vowel harmony and a consonant harmony co-occur within the same language. For example, `apapp`, `abbap` and `popooo` are good, while `apabo` and `appoppp` are not.

In [3]:
def vc_harmony_generate(n = 10, length = 10, grammatical = True,
                       vowels = None, consonants = None):
    """ Generates a collection words following the rule of vowel-consonant harmony.
    
    * n (int): number of strings that need to be generated;
    * length (int): length of every one of the generated strings;
    * grammatical (bool): if set to True, the correctly harmonizing
                          forms are generated, and if set to False,
                          the disharmonic forms are produced;
    * vowels (list): list of vowels among which the agreement
                     is established;
    * consonants (list): list of consonants among which the agreement
                         is established.
    """
    
    # initialization and sanity check for the list of vowels
    if vowels is None:
        vowels = ["a", "o"]
    elif len(vowels) < 2:
        raise IndexError("The vowel system needs to contain at least two distinct vowels.")
        
    # initialization and sanity check for the list of consonants
    if consonants is None:
        consonants = ["p", "b"]
    elif len(consonants) < 2:
        raise IndexError("The consonant system needs to contain at least two distinct consonants.")
    elif [i for i in vowels if i in consonants] or [i for i in consonants if i in vowels]:
        raise ValueError("Lists of harmonizing vowels and transparent elements cannot overlap.")
        
    # generate the required number of harmonic strings
    strings = []
    for i in range(n):
        v = choice(vowels)
        c = choice(consonants)
        new = "".join([choice([v, c]) for j in range(length)])
        
        # the ungrammatical forms are created by taking a random index
        # and rewriting it to the opposite vowel / consonant
        if not grammatical:
            ind = choice(range(length))
            if new[ind] == c:
                new = new[:ind] + choice([i for i in consonants if i != c]) + new[ind + 1:]
            else:
                new = new[:ind] + choice([i for i in vowels if i != v]) + new[ind + 1:]
                
        strings.append(new)
    
    return strings

### Multi-tier Input-sensitive harmony

The trigger for a long-distance assimilation depends on a local context. For example, `e` immediately before `x` prohibits `a` anywhere further after `e` in the string. Then `eaaxaae` and `axaexeeexx` are good, while `exxae` is not.

#### 1. Preparing a class to encode input sensitive rules

In [4]:
class SSRule(object):
    """ A generic template for a input-sensititve rule. 
    
    * symbols (tuple): list of tier symbols relevant for the generalization;
    * target (str): a target character context of which is important;
    * right_context (str): a context in which a target character
                           is projected on the tier;
    * can_follow (tuple): a list of tier symbols that are allowed after
                         the target character is projected.
    """
    def __init__(self, symbols, target, right_context, can_follow):
        self.symbols = symbols
        self.target = target
        self.right_context = right_context
        self.can_follow = can_follow

    def is_grammatical(self, string):
        """ Checks if the given form follows a rule that is encoded.
        
        * string (str): a string well-formedness of which needs to be checked.
        """
        
        # get rid of all irrelevant symbols (not symbols and contexts)
        string = "".join([i for i in string if i in list(self.symbols) + [self.right_context]])
        
        # construct a tier of that strings
        tier = ""
        for i in range(len(string)):
            if string[i] in self.symbols:
                if string[i] == self.target and i < len(string) - 1 and\
                    string[i + 1] == self.right_context:
                    tier += self.target
                elif string[i] != self.target:
                    tier += string[i]

        # check if that tier is well-formed
        for t in range(len(tier)):
            if tier[t] == self.target and t < len(tier) - 1 and\
                tier[t + 1] not in self.can_follow:
                return False
        return True

#### 2. Writing a generator of a sequence grammatical wrt the rule

In [5]:
def generate_rule_sequence(rule, length = 7, grammatical = True):
    """ This function generates a sequence of symbols (un)grammatical 
        with respect to the given rule.
        
    * rule (SSRule): a rule describing an input sensitive dependency;
    * length (int): length of the generated sequence;
    * grammatical (bool): produces correct form when set to True, and 
                          makes a mistake when set to False.
    """
    
    # the generation of the well-formed sequence is done by a simplistic FSA
    sequence = ""
    state = 0
    for i in range(length):
        
        # State 0: the target was not observed
        if state == 0:
            sequence += choice(list(rule.symbols) + [rule.right_context])
            if sequence[-1] == rule.target:
                state = 1
                
        # State 1: the target was observed
        elif state == 1:
            sequence += choice(list(rule.symbols) + [rule.right_context])
            if sequence[-1] == rule.right_context:
                state = 2
            elif sequence[-1] != rule.target:
                state = 0
                
        # State 2: the right context was observed
        elif state == 2:
            sequence += choice(list(rule.can_follow) + [rule.right_context])
                
    # if the ungrammatical form is needed, a violating sequence is generated
    # and inserted into a random position within the sequence
    if not grammatical:
        violate = rule.target + rule.right_context +\
            choice([i for i in list(rule.symbols) if i not in rule.can_follow])
        index_violate = choice(range(length - 3))
        sequence = sequence[:index_violate] + violate + sequence[index_violate + 3:]
        
    return sequence

#### 3. Intertwine

In [6]:
def intertwine(str1, str2, r = (0, 3)):
    """ Intertwines two strings: str1 and str2. At every step, it takes
    some characters from one string, and then some characters from another.
    oxxooxa
    * str1 (str): the first string;
    * str2 (str): the second string;
    * r (tuple[int, int]): min and max+1 symbols to be taken.
    """
    new_string = ""
    current = choice([1, 2])
    while str1 or str2:
        if current == 1:
            cut = choice(range(r[0], r[1]))
            if len(str1) < cut:
                new = str1[:]
            else:
                new = str1[:cut]
            new_string += new
            str1 = str1[len(new):]
            current = 2
        elif current == 2:
            cut = choice(range(r[0], r[1]))
            if len(str2) < cut:
                new = str2[:]
            else:
                new = str2[:cut]
            new_string += new
            str2 = str2[len(new):]
            current = 1
    return new_string

#### 4. Generator for the ITSL harmony
A single locally-driven long-distance assimilation.

In [7]:
def itsl_harmony_generate(n = 10, length = 10, grammatical = True,
                       rule_1 = None):
    """ Generates a collection words following the given rules of the structure-
        Takessensitive dependencies that involve a single tier.
    
    * n (int): number of strings that need to be generated;
    * length (int): length of every one of the generated strings;
    * grammatical (bool): if set to True, the correctly harmonizing
                          forms are generated, and if set to False,
                          the disharmonic forms are produced;
    * rule_1 (SSRule): the first rule describing a long-distant structure-
                      sensitive dependency.
    """
    
    # set the first rule
    if rule_1 == None:
        rule_1 = SSRule(symbols = ("o", "e", "a"), target = "o",\
                        right_context = "x", can_follow = ("a", "o"))
    strings = []
    for i in range(n):
        string = generate_rule_sequence(rule_1, length)
        if not grammatical:
            string = generate_rule_sequence(rule_1, length, grammatical = False)
        strings.append(string)
    return strings

#### 5. Generator for the MITSL harmony
Two locally-driven long-distance assimilations.

In [8]:
def imtsl_harmony_generate(n = 10, length = 10, grammatical = True,
                       rule_1 = None, rule_2 = None):
    """ Generates a collection words following the given rules of the structure-
        Takesinput sensitive dependencies that involve several tiers.
    
    * n (int): number of strings that need to be generated;
    * length (int): length of every one of the generated strings;
    * grammatical (bool): if set to True, the correctly harmonizing
                          forms are generated, and if set to False,
                          the disharmonic forms are produced;
    * rule_1 (SSRule): the first rule describing a long-distant input-
                      sensitive dependency;
    * rule_2 (SSRule): the second rule describing a long-distant input-
                      sensitive dependency.
    """
    
    # set both rules
    if rule_1 == None:
        rule_1 = SSRule(symbols = ("o", "e", "a"), target = "o",\
                        right_context = "x", can_follow = ("a", "o"))
    if rule_2 == None:
        rule_2 = SSRule(symbols = ("b", "p", "d"), target = "b",\
                        right_context = "y", can_follow = ("b", "p"))
    
    strings = []
    for i in range(n):
        # generate two tiers independently, and then intertwine them
        # WARNING: the tier alphabets of the two rules cannot overlap
        #          (required by both learner and generator)
        len_part_1 = length // 2
        len_part_2 = length - len_part_1

        part_1 = generate_rule_sequence(rule_1, len_part_1)
        part_2 = generate_rule_sequence(rule_2, len_part_2)

        if not grammatical:
            mistake = choice(["R1", "R2", "both"])
            if mistake == "R1":
                part_1 = generate_rule_sequence(rule_1, len_part_1, grammatical = False)
            elif mistake == "R2":
                part_2 = generate_rule_sequence(rule_2, len_part_2, grammatical = False)
            else:
                part_1 = generate_rule_sequence(rule_1, len_part_1, grammatical = False)
                part_2 = generate_rule_sequence(rule_2, len_part_2, grammatical = False)

        # intertwining the two generated sequences
        new_string = intertwine(part_1, part_2)
        strings.append(new_string)
        
    return strings

### Tools: collecting data generators for the experiments

In [9]:
def generate_harmony(kind="first-last", length=range(2, 7), number=1000):
    """
    Generates a harmony based on 3 parameters.
    
    Arguments:
    * kind (str): type of the harmony, choices:
        "first-last", "double", "assimilation-one", "assimilation-two"
    * length (range): a range of lengths of the intended strings
    * number (int): a number of strings to be generated
    
    Outputs:
    * list: a collection of strings.
    """
    
    # preparing data for easy generation
    lennum = {r:number // len(length) for r in length}
    hmap = {"first-last" : first_last_generate,
                   "double" : vc_harmony_generate,
                   "assimilation-one" : itsl_harmony_generate,
                   "assimilation-two" : imtsl_harmony_generate}
    
    # generating the data
    data = [i for l in lennum for i in hmap[kind](lennum[l], l)]
    
    # annotate the data with start- and end-markers >> and <<
    return list(map(lambda string : ">>" + string + "<<", data))

## Code of the learning algorithm

### Generating only well-formed n-grams

First, since we generate ngrams automatically, we need to have a checker that nothing like `a<<b`, `a><k`, or `fdh<` is generated.

In [10]:
def _well_formed_ngram(ngram):
    """ Checks if ngram is well-formed. """
    
    start, end = [], []
    for i in range(len(ngram)):
        if ngram[i] == ">":
            start.append(i)
        elif ngram[i] == "<":
            end.append(i)

    start_len, end_len = len(start), len(end)
    if any([start_len == len(ngram), end_len == len(ngram)]):
        return False
    
    if start_len > 0:
        if ngram[0] != ">":
            return False
        if start_len > 1:
            for i in range(1, start_len):
                if start[i] - start[i - 1] != 1:
                    return False
    
    if end_len > 0:
        if ngram[-1] != "<":
            return False
        if end_len > 1:
            for i in range(1, end_len):
                if end[i] - end[i - 1] != 1:
                    return False
                
    # this part is different from the checker from the SigmaPie
    # to avoid passing fourgrams such as "><<<"
    if len(ngram) > 3 and (ngram[1] == "<" or ngram[2] == ">"):
        return False

    return True

In [11]:
def generate_all_ngrams(alphabet, size):
    """ To generate all possible ngrams based on the given alphabet. """
    
    all_of_them = ["".join(i) for i in product(alphabet + ["<", ">"], repeat = size)]
    return [i for i in all_of_them if _well_formed_ngram(i)]

In [12]:
def unattested_ngrams(data, all_ngrams):
    """ Detect list of ngrams that is unattested in the given data. """
    not_in_data = []
    for f in all_ngrams:
        found = False
        for string in data:
            if f in string:
                found = True
                break
        if not found:
            not_in_data.append(f)
            
    return not_in_data

In [13]:
def string_paths(string, fourgram):
    """ Finds all paths for two given symbols and their contexts. """
    
    paths = {}
    
    ind_first = [s.start() for s in re.finditer(fourgram[:2], string)]
    ind_second = [s.start() for s in re.finditer(fourgram[2:], string)]
    
    if not (ind_first and ind_second):
        return []
    
    paths = set()
    
    for f in ind_first:
        for s in ind_second:
            if f >= s:
                continue
                
            middle = string[f+1:s+1]
            in_between = tuple(set(middle[i:i+2] for i in range(len(middle) - 1)))
            path = (fourgram[:2], in_between, fourgram[2:])
            paths.add(path)

    return paths

In [14]:
def data_paths(data, fourgram):
    """ Collects a list of paths for a particular (unattested) pair of bigrams. """
    
    total_paths = set()
    for d in data:
        total_paths = total_paths.union(string_paths(d, fourgram))
        
    return total_paths


def find_relevant_paths(data, unattested_ngrams):
    """ Finds all paths for the unattested ngrams. """
    
    relevant_paths = dict()
    for un in unattested_ngrams:
        relevant_paths[un] = data_paths(data, un)
        
    return relevant_paths

In [15]:
def learn_imtsl(data, alphabet, n = 2, context = 2, redacted=False):
    """
    A function that extracts 2local MITSL2 grammars.
    
    Arguments:
    data (list): examples from the target language;
    alphabet (list or set): symbols of the language;
    n (int): size of the target ngrams (available for 2);
    context (int): size of the local context considered (available for 2);
    redacted (bool): if set to False, shows grammar as
                        {single restriction : corresponding tier alphabet},
                     if set to True, shows grammar as
                        {tier alphabet : corresponding restrictions}.
    
    Returns:
    dict: keys are tier alphabets, values are tier grammars.
    """
    
    if n != 2 or context != 2:
        raise NotImplementedError("This algorithm does not support such values yet.")
        
    # collect a list of unattested ngrams (n * context = 2 * 2 = 4)
    unattested = unattested_ngrams(data, generate_all_ngrams(alphabet, 4))
    
    # build a look-up table with the relevant paths
    relevant_paths = find_relevant_paths(data, unattested)
    
    # initialize the grammar and the maximum tier
    max_tier_guess = generate_all_ngrams(alphabet, context)
    grammar = dict()
    
    for un in unattested:
        
        # collect the collection of paths for the unattested bigrams
        local_tier = [un[:2], un[2:]]
        local_relevant = [(i[0], set(i[1]), i[2]) for i in relevant_paths[un]]
        
        for ss in max_tier_guess:
            
            # members of the unattested bigram must be on the tier
            if ss in [un[:2], un[2:]]:
                continue
            
            # if can remove from any path, the symbol is not a tier symbol
            contain_ss = [i for i in local_relevant if ss in i[1]]
            added = False
            for pth in contain_ss:
                no_ss_in_path = (pth[0], set(i for i in pth[1] if i != ss), pth[2])
                if no_ss_in_path not in local_relevant:
                    local_tier.append(ss)
                    added = True
                    break
                if added:
                    continue
        
        # assemble the grammar
        grammar[(un[:2], un[2:])] = local_tier[:]
        
    if not redacted:
        return grammar
    
    # if redacted=True, reassemble the grammar
    new_grammar = dict()
    for pair in grammar:
        if tuple(grammar[pair]) not in new_grammar:
            new_grammar[tuple(grammar[pair])] = [pair]
        else:
            new_grammar[tuple(grammar[pair])].append([pair])
    
    del grammar
    return new_grammar

## Experiments

In [16]:
fl_harmony = generate_harmony("first-last", range(2, 8), number = 10000)
fl_sigma = ["a", "x", "o"]

fl_grammar = learn_imtsl(fl_harmony, fl_sigma)
# pprint(fl_grammar)

In [17]:
double_harmony = generate_harmony("double", range(2, 8), number = 10000)
double_sigma = ["a", "o", "b", "p"]

double_grammar = learn_imtsl(double_harmony, double_sigma)
# pprint(double_grammar)

In [18]:
assim_one = generate_harmony("assimilation-one", range(2, 8), number = 10000)
assim_one_sigma = ["a", "o", "e", "x"]

assim_one_grammar = learn_imtsl(assim_one, assim_one_sigma)
# pprint(assim_one_grammar)

for i in assim_one_grammar:
    if i[0] == 'ox':
        print(i, ":", assim_one_grammar[i])

('ox', 'ae') : ['ox', 'ae']
('ox', 'oe') : ['ox', 'oe']
('ox', 'ea') : ['ox', 'ea']
('ox', 'eo') : ['ox', 'eo']
('ox', 'ee') : ['ox', 'ee']
('ox', 'ex') : ['ox', 'ex']
('ox', 'e<') : ['ox', 'e<']
('ox', 'xe') : ['ox', 'xe']


In [19]:
assim_two = generate_harmony("assimilation-two", range(2, 8), number = 10000)
assim_two_sigma = ["a", "o", "e", "x", "b", "p", "d", "y"]

assim_two_grammar = learn_imtsl(assim_two, assim_two_sigma)
# pprint(assim_two_grammar)

for i in assim_two_grammar:
    if i[0] in {'ox', 'by'}:
        print(i, ":", assim_two_grammar[i])

('ox', 'aa') : ['ox', 'aa']
('ox', 'ao') : ['ox', 'ao']
('ox', 'ae') : ['ox', 'ae']
('ox', 'ax') : ['ox', 'ax']
('ox', 'oa') : ['ox', 'oa']
('ox', 'oo') : ['ox', 'oo']
('ox', 'oe') : ['ox', 'oe']
('ox', 'ox') : ['ox', 'ox']
('ox', 'ea') : ['ox', 'ea']
('ox', 'eo') : ['ox', 'eo']
('ox', 'ee') : ['ox', 'ee']
('ox', 'ex') : ['ox', 'ex']
('ox', 'eb') : ['ox', 'eb']
('ox', 'ep') : ['ox', 'ep']
('ox', 'ed') : ['ox', 'ed']
('ox', 'ey') : ['ox', 'ey']
('ox', 'e<') : ['ox', 'e<']
('ox', 'xa') : ['ox', 'xa']
('ox', 'xo') : ['ox', 'xo']
('ox', 'xe') : ['ox', 'xe']
('ox', 'xx') : ['ox', 'xx']
('ox', 'be') : ['ox', 'be']
('ox', 'pe') : ['ox', 'pe']
('ox', 'de') : ['ox', 'de']
('ox', 'ye') : ['ox', 'ye']
('by', 'ad') : ['by', 'ad']
('by', 'od') : ['by', 'od']
('by', 'ed') : ['by', 'ed']
('by', 'xd') : ['by', 'xd']
('by', 'bd') : ['by', 'bd']
('by', 'pd') : ['by', 'pd']
('by', 'da') : ['by', 'da']
('by', 'do') : ['by', 'do']
('by', 'de') : ['by', 'de']
('by', 'dx') : ['by', 'dx']
('by', 'db') : ['by'